In [1]:
import datetime
import pandas as pd
import time
import requests
import os
import numpy as np
import xlsxwriter
from abc import ABCMeta, abstractmethod
import Queue
from collections import deque
import getpass
import pandas.io.formats.excel

In [2]:
# TODOs:

# PNL reporting/collecting relevant statistics for visualizations from our simulations

# Longer time horizons. Only weight according to n most recent time steps

# Benchmarks: 
## - Buy and hold market index (Dow Jones)
## - Constantly Rebalanced Portfolio (maintain equal weights of each asset at all times)

# Non-fractional shares

# Contexts

# Other trading strategies
## Fundamentals-based strategies (need to get more data for this)



### Simulation runner

In [12]:

class SimulationEnv(object):
    
    loggables = ['positions']
    
    def __init__(self, wealth, assets, 
                 path_to_data, start_date, end_date,
                 agent_type, expert_type, reinvest):

        self.init_wealth = wealth
        self.wealth = wealth
        self.assets = assets
        self.agent_type = agent_type
        self.expert_type = expert_type
        self.path_to_data = path_to_data
        self.reinvest = reinvest
        ## TODO: 
        ## - specify start and end date for simulation
        ## - collect statistics on simulation results
        
        return
    
    
    ## To be called before running a simulation - initialize experts, agents, and initial position, etc.

    def setup_params(self, agent_args={}, expert_args={}):
    
        self.experts = [self.expert_type(a, self.path_to_data, **expert_args) for a in self.assets]
        self.agent = self.agent_type(self.experts, **agent_args)
        self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
        
        ## TODO: 
        # do various other stuff here like for select for high-volatility stocks or something
        # exception handling
        # Need to make sure data files are in sync
        
        return
        
    ## Run simulation
    def run(self, log=False, logpath=os.getcwd()):
        
        ## Start period counter and log
        self.period = 1
        self.finallog = []
        self.log = log
        self.logpath = logpath
        
        ## Warmup period: 
        ## i.e. for strategies involving moving average indicators, wait until we have enough data to calculate MA
        for expert in self.agent.experts:
            expert.warmup()
        
        ## Simulation: Go until data iterators reach the end
        while True:
            try:

                ## Log this period
                if log:
                    self.logperiod()
                    
                ## Update experts with last period's rewards
                for i, expert in enumerate(self.agent.experts):
                    expert.update()
                    if self.reinvest:# below is to redistribute unallocated money
                        if expert.pick == False:
                            extra_money = self.positions[i]
                            self.positions[i] = 0
                            distribute = np.greater(self.positions, 0)
                            if not np.any(distribute): # this is to avoid divide by zero if nothing is invested
                                #print("all experts say not to buy")
                                self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
                            else:
                                distribute_extra_money = distribute * (extra_money/ np.sum(distribute))
                                self.positions = np.add(self.positions, distribute_extra_money)
                            assert(abs(np.sum(self.positions)-self.wealth) <= 1)# make sure all money is accounted for
                      
                ## Update agent accordingly (i.e. for Hedge, update weights according to each expert's reward in the last period)
                self.agent.update()
                

                
                ## Update position with returns from last period
                self.positions = self.positions * (1 + self.agent.returns)
                self.wealth = np.sum(self.positions)
                
                ## Rebalance according to new, updated weights
                ## TODO: only allow non-fractional shares to be purchased (?)
                if np.sum(self.agent.weights) == 1:
                    self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
                    
                ## Advance period
                self.period += 1
                
            except StopIteration:
                break
        
        ## Write the log file
        if log:
            self.savelog()

    def logperiod(self):
        row = [self.period] + [self.wealth]
        nrow = []
        for loggable in (self.loggables):
            if getattr(self,loggable) is None:
                nrow += [None] * len(self.experts)
            else:
                nrow += getattr(self,loggable).tolist()
        arow = []
        for loggable in (self.agent_type.loggables):
            if getattr(self.agent,loggable) is None:
                arow += [None] * len(self.experts)
            else:
                arow += getattr(self.agent,loggable).tolist()
        erow = []
        for loggable in (self.expert_type.loggables):
            if [getattr(e,loggable) for e in self.experts] is None:
                erow += [None] * len(self.experts)
            else:
                erow += [getattr(e,loggable) for e in self.experts]
        row += nrow + arow + erow
        self.finallog.append(row)
    
    def savelog(self):
        runtime = datetime.datetime.now()
        runuser = getpass.getuser()
        logname = runuser + "_" + runtime.strftime('%Y-%m-%d_%H-%M-%S')
        if not os.path.exists(os.path.join(self.logpath, logname)):
            os.makedirs(os.path.join(self.logpath, logname))
        col = ['period', 'wealth'] + \
            [x+'.'+y for x in self.loggables for y in self.assets] + \
            [x+'.'+y for x in self.agent_type.loggables for y in self.assets] + \
            [x+'.'+y for x in self.expert_type.loggables for y in self.assets]
        df = pd.DataFrame(self.finallog, columns=col)
        df.set_index('period', inplace=True)
        
        
        writer = pd.ExcelWriter(os.path.join(self.logpath, logname, logname+'.xlsx'), engine='xlsxwriter')
        pd.io.formats.excel.header_style = None
        df.to_excel(writer,'run_log')
        
        workbook  = writer.book
        worksheet = writer.sheets['run_log']
        
        header_format = workbook.add_format({'bold': True,'text_wrap': True})
        worksheet.set_row(0, None, header_format)
        writer.save()



### Base classes

Base classes for agents and experts to be implemented by us

In [4]:

class Agent(object):
    __metaclass__ =ABCMeta
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
    @abstractmethod
    def act(self):
        pass
    
class Expert(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, reward_data):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
class Context(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, context_data):
        pass
    
    @abstractmethod
    def observe(self):
        pass

### Agents

Implementations of different online portfolio selection algorithms
* Exponential Gradient
* Exponential Gradient (recent history only)

In [5]:
## EG. (http://www.cis.upenn.edu/~mkearns/finread/helmbold98line.pdf)
class EG(Agent):
    
    loggables = ['returns','rewards','weights']
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta):
        self.eta = eta
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = np.asarray([e.returns for e in self.experts])
        multipliers = np.exp(self.eta * self.rewards/np.sum(self.weights * self.rewards))
        self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
        self.weights = np.nan_to_num(self.weights)

        return
    
    def act(self):
        return self.weights


class EGRecent(Agent):
    
    loggables = ['returns','rewards','weights']
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta, n):
        self.eta = eta
        self.experts = experts
        #self.weights = Queue(maxsize=n)
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = deque(maxlen=n)
        self.returns = None
        self.t = 0
        self.n = n
        #self.rewards = None
        #self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
           
        self.rewards.append(np.asarray([e.reward for e in self.experts]))
        self.returns = np.asarray([e.returns for e in self.experts])
        
        self.weights = np.ones(len(self.experts))/len(self.experts)
        #print self.t
        #print len(self.rewards)
        for rewards in self.rewards:
            rewards = np.asarray(rewards)
            multipliers = np.exp(self.eta * rewards/np.sum(self.weights * rewards))
            self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
            self.weights = np.nan_to_num(self.weights)
        self.t += 1
        #print self.t
        
        return
    
    def act(self):
        return self.weights
    
      
        

More agents to serve as naive/benchmark portfolio allocation algorithms:
* Equal-weighted buy and hold
* Market-weighted buy and hold
* Constantly rebalanced portfolio


In [6]:
## Some other examples of agents to use as benchmarks 

class NaiveBuyHold(Agent):
    
    loggables = ['rewards','weights']
    
    def __init__(self, experts):
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        return
    
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = self.rewards
        return
    
    def act(self):
        return self.weights
    
## TODO
class WeightedBuyHold(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    

## TODO
class ConstantRebalancer(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return


### Experts

* Dummy
* Mean Reversion
* Momentum

In [7]:

## Dummy expert that always pick the same asset
class Dummy(Expert):
    
    loggables = ['reward']
    
    ## Expert has a reward associated with its pick
    def __init__(self, name, path_to_data):
        self.reward = 0.
        self.pick = True ## Might not be necessary
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        return
    
    ## Expert updates its reward 
    def update(self):
        current_price = float(self.data.get_chunk(1)["adj_close"])
        self.reward = (current_price - self.last_price)/self.last_price
        self.returns = self.reward
        self.last_price = current_price
        return
    
    def warmup(self):
        pass
    

class MeanReversion(Expert):
    
    loggables = ['avg','std']
    
    def __init__(self, name, path_to_data, window_size, threshold):
        self.reward = 0.
        self.pick = False
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        self.window_size = window_size
        self.avg = 0.0
        self.std = 0.0
        self.threshold = threshold
        self.last_n_prices = Queue(maxsize=10)
        self.returns = 0.
        return
    
    def warmup(self):
        n = 1
        while n <= self.window_size:
            self.last_n_prices.put(self.last_price)
            self.last_price = float(self.data.get_chunk(1)["adj_close"])
            n += 1
        return
        
    def update(self):
        _ = self.last_n_prices.get()
            
        self.last_n_prices.put(self.last_price)
        self.avg = np.mean(list(self.last_n_prices.queue))
        self.std = np.std(list(self.last_n_prices.queue))
        
        current_price = float(self.data.get_chunk(1)["adj_close"])

        ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
        if self.pick:
            self.reward = (current_price - self.last_price)/self.last_price
            self.returns = self.reward
        else:
            self.reward = -(current_price - self.last_price)/self.last_price
            self.returns = 0
        self.last_price = current_price

        if self.last_price <= self.avg - self.threshold * self.std:
            self.pick = True
        else:
            self.pick = False

        return
    
    
class Momentum(Expert):
    
    loggables = ['avg','std']
    
    def __init__(self, name, path_to_data, window_size, threshold):
        self.reward = 0.
        self.pick = False
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        self.window_size = window_size
        self.avg = 0.0
        self.std = 0.0
        self.threshold = threshold
        self.last_n_prices = Queue(maxsize=10)
        self.returns = 0.
        return
    
    def warmup(self):
        n = 1
        while n <= self.window_size:
            self.last_n_prices.put(self.last_price)
            self.last_price = float(self.data.get_chunk(1)["adj_close"])
            n += 1
        return
        
    def update(self):
        _ = self.last_n_prices.get()
            
        self.last_n_prices.put(self.last_price)
        self.avg = np.mean(list(self.last_n_prices.queue))
        self.std = np.std(list(self.last_n_prices.queue))
        
        current_price = float(self.data.get_chunk(1)["adj_close"])

        ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
        if self.pick:
            self.reward = (current_price - self.last_price)/self.last_price
            self.returns = self.reward
        else:
            self.reward = -(current_price - self.last_price)/self.last_price
            self.returns = 0
        self.last_price = current_price

        if self.last_price >= self.avg - self.threshold * self.std:
            self.pick = True
        else:
            self.pick = False

        return
  

### Contexts

In [8]:
  
class VolContext(Context):
    
    def __init__(self, data_file, threshold):
        self.data = pd.read_csv(data_file, iterator=True, chunksize=1)
        self.threshold = threshold
        self.contexts = ["HighVol", "LowVol"]
        return
    
    # Returns a string giving the name of the context
    def observe(self):
        if float(self.data.get_chunk(1)["adj_close"]) > self.threshold:
            return self.contexts[0]
        else:
            return self.contexts[1]

In [9]:
stocks = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GE',
 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 
'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM']

In [10]:
initial_wealth = 100000

In [13]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01})
s.run(log=True, logpath="logs")

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

Initial wealth: $100000
Final wealth: $129778.013883
Annualized return: 0.0934824414257
Time: 19s


In [67]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, Dummy)
s.setup_params(agent_args = {"eta": -0.01})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: overflow encountered in exp
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: divide by zero encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in multiply


Initial wealth: $100000
Final wealth: $512772.31064
Annualized return: 0.095528541711
Time: 83s


In [66]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EGRecent, Dummy)
s.setup_params(agent_args = {"eta": -0.01, "n": 100})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: overflow encountered in exp
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:58: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:58: RuntimeWarning: invalid value encountered in multiply
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: invalid value encountered in divide


Initial wealth: $100000
Final wealth: $161839.725895
Annualized return: 0.179468459345
Time: 51s


In [72]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Dummy)
s.setup_params(agent_args = {"eta": -0.01, "n": 750})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: overflow encountered in exp
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:58: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:58: RuntimeWarning: invalid value encountered in multiply


Initial wealth: $100000
Final wealth: $55374.5056234
Annualized return: -0.0324506877373
Time: 825s


In [75]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", NaiveBuyHold, Dummy)
s.setup_params(expert_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))



Initial wealth: $100000
Final wealth: $131862.034286
Annualized return: 0.0994713617583
Time: 20s


In [76]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", NaiveBuyHold, Dummy)
s.setup_params(expert_args={})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

Initial wealth: $100000
Final wealth: $196477.308454
Annualized return: 0.0384149324082
Time: 89s


In [77]:
## Run simulation where we each expert only recommends to buy when the price crosses 0.5 standard deviations below the 10-day moving average. 
## Otherwise doesn't do anything
## Sell position when price gets above 0.5 standard deviations below the 10-day MA

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, MeanReversion)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

Initial wealth: $100000
Final wealth: $148046.711108
Annualized return: 0.143990540447
Time: 22s


In [78]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, MeanReversion)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: overflow encountered in exp
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: divide by zero encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in multiply


Initial wealth: $100000
Final wealth: $763454.648775
Annualized return: 0.120138237954
Time: 93s


In [80]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, MeanReversion)

s.setup_params(
    agent_args={"eta": -0.01, "n": 1250},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: overflow encountered in exp
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:58: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:58: RuntimeWarning: invalid value encountered in multiply


Initial wealth: $100000
Final wealth: $834042.749701
Annualized return: 0.125680548297
Time: 206s


In [93]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, Momentum)

years = 2. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

Initial wealth: $100000
Final wealth: $139211.576888
Annualized return: 0.120108572408
Time: 21s


In [94]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, Momentum)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: overflow encountered in exp
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: divide by zero encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in multiply
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in divide


Initial wealth: $100000
Final wealth: $172374.2603
Annualized return: 0.0308570856091
Time: 89s


In [88]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Momentum)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.001, "n": 500},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: overflow encountered in exp
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:58: RuntimeWarning: invalid value encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in divide
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:58: RuntimeWarning: invalid value encountered in multiply
/Users/kwu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: invalid value encountered in divide


Initial wealth: $100000
Final wealth: $188378.253748
Annualized return: 0.0359780514752
Time: 129s


In [144]:
pd.__version__

u'0.20.1'